In [ ]:
# 데이터 수집

import pandas as pd

df_info = pd.read_csv('info_dataset/info.xlsx', encoding='utf8', index_col=0, errors='ignore')
df_0 = pd.read_csv('review_dataset/reveiw0.csv', encoding='utf8', index_col=0, errors='ignore')
df_1 = pd.read_csv('review_dataset/reveiw1.csv', encoding='utf8', index_col=0, errors='ignore')
df_2 = pd.read_csv('review_dataset/reveiw2.csv', encoding='utf8', index_col=0, errors='ignore')
df_3 = pd.read_csv('review_dataset/reveiw3.csv', encoding='utf8', index_col=0, errors='ignore')
df_4 = pd.read_csv('review_dataset/reveiw4.csv', encoding='utf8', index_col=0, errors='ignore')
df_5 = pd.read_csv('review_dataset/reveiw5.csv', encoding='utf8', index_col=0, errors='ignore')
df_6 = pd.read_csv('review_dataset/reveiw6.csv', encoding='utf8', index_col=0, errors='ignore')
df_7 = pd.read_csv('review_dataset/reveiw7.csv', encoding='utf8', index_col=0, errors='ignore'
df_8 = pd.read_csv('review_dataset/reveiw8.csv', encoding='utf8', index_col=0, errors='ignore')

df_0.head()

In [ ]:
# 크롤링 방법

from bs4 import BeautifulSoup
import pandas as pd

## hotel_review_list.csv 를 읽어와서 데이터프레임을 만드는 함수
def make_df(url_list, num) :

    columns = url_list.iloc[:,num].dropna()
    
    for idx,column in enumerate(columns) :
        columns[idx] = column.split('/')[-1] # url의 고유 ID 가져오기

        df = pd.DataFrame(columns=columns)

    return df

## 소스를 받으면 리뷰 내용들을 찾아서 하나의 문자열로 합친 후 반환하는 함수
def get_reviews(src, df, col) : # src : 소스 , df : 저장될 데이터 프레임, col : 호텔 ID 컬럼
    # print(df)
    col_name = df.columns[col]
    idx = 0
    soup = BeautifulSoup(src, 'lxml')

    review_list_short = soup.find_all('p',attrs={"class":"content"})        # 짧은 리뷰 리스트
    review_list_long = soup.find_all('p',attrs={"class":"content clamp"})   # 더보기 리뷰 리스트

    # 리뷰를 df에 저장 
    print('짧은 리뷰 저장 중')
    for i in range(len(review_list_short)) :     
        df.loc[idx, col_name] = review_list_short[i].get_text()
        idx += 1
        # print('.',end='')

    print('\n긴 리뷰 저장 중')
    for i in range(len(review_list_long)) :
        df.loc[idx, col_name] = review_list_long[i].get_text()   
        idx += 1
        # print('.',end='')

    return df

In [ ]:
# 리뷰 추출

from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

import method       # method.py가 같은 폴더에 있어야 한다.

## 여기 이전에 했던 거처럼 0~8번 자동으로 넣어주게 해도 좋아요. 이거는 숫자 일일이 넣어야 됩니다.
num = int(input('input number : '))

# url csv 파일 읽어오는 부분
url_list = pd.read_csv('hotel_review_list.csv', index_col = 0)

# 각 컬럼의 길이 구하기
numofhotel = url_list.iloc[:,num].notnull().sum()

# 내보낼 df 생성 (row : 리뷰, col : ID)
# ID : 호텔 url 맨 뒤 숫자부분
df = method.make_df(url_list, num)

# 입력된 column 의 url을 훑으며 크롤링 시작
for idx in range(numofhotel) :

    # 진행상황 출력
    print('-'*60)
    print(f'read page {idx}/{numofhotel}')

    # '/reviews' 를 붙여 url 완성
    url = url_list.iloc[idx,num] + '/reviews' 
    print(f'url : {url}') 

    # chromedriver 옵션 설정
    options = Options()
    user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    options.add_argument('user-agent=' + user_agent)    # 유저의 접속인 것처럼 속이기 
    options.add_argument("headless")        # 창 띄우고 싶으면 주석처리.    

    # chromedriver 실행 후 url 정보 받기
    driver = webdriver.Chrome(options=options, executable_path='chromedriver')
    driver.get(url = url)

    # 스크롤 맨 밑까지 내리는 작업
    print('스크롤 내리는 중')
    while True :
        print('.',end='')
        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
       
        time.sleep(2)        ## 스크롤 내리는 로딩시간이 길면 숫자 좀 늘려주세요!

        new_height = driver.execute_script("return document.body.scrollHeight")        

        if new_height == last_height :
            break        
    print('')

    # 크롤링 시작
    print('start crawling...')
    src = driver.page_source    # 소스를 가져와서 리뷰 데이터 긁어올 것임    
    
    # 데이터프레임에 크롤링 정보 저장
    df = method.get_reviews(src, df, idx) # method 파일 참조하여 리뷰 수집
    print('done!')

    ## tmp
    if idx % 10 == 1 :
        df.to_csv(f'./reveiw_list{num}.csv', encoding="utf-8-sig")    
        print('tmp saved')    

# 모든 작업 종료 후 파일 내보내기
df.to_csv(f'./reveiw_list{num}.csv', encoding="utf-8-sig")
print('Complete!!')

In [ ]:
# 데이터 정제

import pandas as pd
import numpy as np

num = 0
for num in range(9) :

    raw_df = pd.read_csv(f'review_dataset/reveiw_list{num}.csv',index_col=0)
    new_df = pd.DataFrame(columns=['hotel','star','review'])

    ids = raw_df.columns
    for id in ids :    
        reviews = raw_df.loc[:,id]

        for review in reviews :
            if pd.isnull(review) :
                break

            new_df.loc[len(new_df), 'hotel'] = id
            new_df.loc[len(new_df) - 1, 'review'] = review
            
        print(f'{num}번 파일 id : {id} done!')

    new_df.to_csv(f'./revised_review{num}.csv', encoding="utf-8-sig")
    print(f'reveiw_list{num}.csv DONE!' )
    print('='*50)